In [1]:
#import libraries
import time
import numpy as np
import pandas as pd
from scipy.io import loadmat
import csv
from sklearn.cluster import DBSCAN
import pyqtgraph as pg
from pyqtgraph.Qt import QtCore, QtGui
%gui qt5
import KMedoids
from sklearn.metrics.pairwise import pairwise_distances

### Plotting 

In [2]:
#set up plottig GUI
app = QtGui.QApplication([])
pg.setConfigOption('background','w')

In [4]:
win = pg.GraphicsWindow(title="Occupancy Detection GUI")
plot1 = win.addPlot()
plot1.setXRange(-6,6)
plot1.setYRange(0,6)
plot1.setLabel('left',text = 'Y position (m)')
plot1.setLabel('bottom', text= 'X position (m)')
s1 = plot1.plot([],[],pen=None,symbol='o')
plot2 = win.addPlot()
plot2.setXRange(-6,6)
plot2.setYRange(0,6)
plot2.setLabel('left',text = 'Y position (m)')
plot2.setLabel('bottom', text= 'X position (m)')
s2 = plot2.plot([],[],pen=None,symbol='o')

### Parsing

In [5]:
def validateChecksum(recieveHeader):
    h = recieveHeader.view(dtype=np.uint16)
    a = np.array([sum(h)], dtype=np.uint32)
    b = np.array([sum(a.view(dtype=np.uint16))], dtype=np.uint16)
    CS = np.uint16(~(b))
    return CS

def readHeader(recieveHeader):
    headerContent = dict()
    index = 0
    
    headerContent['magicBytes'] = recieveHeader[index:index+8]
    index += 20
    
    headerContent['packetLength'] = recieveHeader[index:index+4].view(dtype=np.uint32)
    index += 4
        
    headerContent['frameNumber'] = recieveHeader[index:index+4].view(dtype=np.uint32)
    index += 24
    
    headerContent['numTLVs'] = recieveHeader[index:index+2].view(dtype=np.uint16)
    
    return headerContent

def tlvParsing(data, dataLength, tlvHeaderLengthInBytes, pointLengthInBytes, targetLengthInBytes):
    
    targetDict = dict()
    pointCloud = None
    index = 0
    #tlv header parsing
    tlvType = data[index:index+4].view(dtype=np.uint32)
    tlvLength = data[index+4:index+8].view(dtype=np.uint32)
    #TLV size check
    if (tlvLength + index > dataLength):
        print('TLV SIZE IS WRONG')
        lostSync = True
        return
    
    index += tlvHeaderLengthInBytes
    pointCloudDataLength = tlvLength - tlvHeaderLengthInBytes
    if tlvType == 6: #point cloud TLV
        numberOfPoints = pointCloudDataLength/pointLengthInBytes
        if numberOfPoints > 0:
            p = data[index:index+pointCloudDataLength[0]].view(dtype=np.single)
            #form the appropriate array 
            #each point is 16 bytes - 4 bytes for each property - range, azimuth, doppler, snr
            pointCloud = np.reshape(p,(4, int(numberOfPoints)),order="F")
    
    return pointCloud

def liveParsing(tlvStream):
    
    tlvHeaderLengthInBytes = 8
    pointLengthInBytes = 16
    frameNumber = 0
    
    tlvStream = np.frombuffer(tlvStream, dtype = 'uint8')
    #tlv header
    index = 0
    #tlv header parsing
    tlvType = tlvStream[index:index+4].view(dtype=np.uint32)
    tlvLength = tlvStream[index+4:index+8].view(dtype=np.uint32)

    index += tlvHeaderLengthInBytes
    tlvDataLength = tlvLength - tlvHeaderLengthInBytes

    if tlvType == 6: 
        numberOfPoints = tlvDataLength/pointLengthInBytes
        p = tlvStream[index:index+tlvDataLength[0]].view(np.single)
        pointCloud = np.reshape(p,(4, int(numberOfPoints)),order="F")

        if not(pointCloud is None):
            #constrain point cloud to within the effective sensor range
            #range 1 < x < 6
            #azimuth -50 deg to 50 deg
            #check whether corresponding range and azimuth data are within the constraints

            effectivePointCloud = np.array([])
            for index in range(0, len(pointCloud[0,:])):
                if (pointCloud[0,index] > 1 and pointCloud[0,index] < 6) \
                and (pointCloud[1, index] > -50*np.pi/180 \
                    and pointCloud[1, index] < 50*np.pi/180):

                    #concatenate columns to the new point cloud
                    if len(effectivePointCloud) == 0:
                        effectivePointCloud = np.reshape(pointCloud[:, index], (4,1), order="F")
                    else:
                        point = np.reshape(pointCloud[:, index], (4,1),order="F")
                        effectivePointCloud = np.hstack((effectivePointCloud, point))

            if len(effectivePointCloud) != 0:
                posX = np.multiply(effectivePointCloud[0,:], np.sin(effectivePointCloud[1,:]))
                posY = np.multiply(effectivePointCloud[0,:], np.cos(effectivePointCloud[1,:]))
                SNR  = effectivePointCloud[3,:]
                return posX, posY, SNR
            
    return None, None, None

### Clustering

In [6]:
def DBSCANOnlyClusteringWeightedSum(pointsX, pointsY):
    
    #initialize constraints/variables
    minClusterSize = 1
    xMean = np.array([])
    yMean = np.array([])
    
    if len(pointsX) >= minClusterSize:

        clusterer = DBSCAN(eps=1, min_samples=5)
        
        clusterer.fit(pd.DataFrame(np.transpose(np.array([pointsX,pointsY]))).values)

        if clusterer.core_sample_indices_.size > 0:
            #array that contains the x,y positions and the cluster association number
            clusters = np.array([pointsX[clusterer.core_sample_indices_],
                      pointsY[clusterer.core_sample_indices_], 
                     clusterer.labels_[clusterer.core_sample_indices_]])
            for centroidNumber in np.unique(clusters[2,:]):
                xMean = np.append(xMean, np.mean(clusters[0,:][np.isin(clusters[2,:], centroidNumber)]))
                yMean = np.append(yMean, np.mean(clusters[1,:][np.isin(clusters[2,:], centroidNumber)]))
                
        

    return yMean, xMean

In [7]:
def DBSCANOnly(pointsX, pointsY, eps, minSamples):
    
    #initialize constraints/variables
    minClusterSize = 1
    clusters = np.array([])
    
    if len(pointsX) >= minClusterSize:

        clusterer = DBSCAN(eps=eps, min_samples=minSamples)
        
        clusterer.fit(pd.DataFrame(np.transpose(np.array([pointsX,pointsY]))).values)

        if clusterer.core_sample_indices_.size > 0:
            #array that contains the x,y positions and the cluster association number
            clusters = np.array([pointsX[clusterer.core_sample_indices_],
                      pointsY[clusterer.core_sample_indices_], 
                     clusterer.labels_[clusterer.core_sample_indices_]])
                
            

    return clusters

### Tracking

In [8]:
def predict(x, P, A, Q): #predict function
    xpred = np.matmul(A,x)
    Ppred = np.matmul(A,P)
    Ppred = np.matmul(Ppred,np.transpose(A)) + Q
    return(xpred, Ppred)

def innovation(xpred, Ppred, z, H, R): #innovation function
    nu = z - np.matmul(H,xpred)
    S = np.matmul(H,Ppred)
    S = R + np.matmul(S, np.transpose(H))
    return(nu, S)

def innovation_update(xpred, Ppred, nu, S, H):
    K = np.matmul(Ppred, np.transpose(H))
    K = np.matmul(K,np.linalg.inv(S)) #check inverse function
    xnew = xpred + np.matmul(K,nu)
    Pnew = np.matmul(K,S)
    Pnew = Ppred - np.matmul(Pnew,np.transpose(K)) 
    return(xnew, Pnew)

def cart2pol(x, y):
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    return(rho, phi)

def data_associate(centroidPred, rthetacentroid):
    rthetacentroidCurrent = rthetacentroid
    centpredCol = np.size(centroidPred,1)
    rthetaCol = np.size(rthetacentroid,1)

    for i in list(range(0,centpredCol)):
        r1 = centroidPred[0][i]
        r2 = rthetacentroid[0]
        theta1 = centroidPred[2][i]
        theta2 = rthetacentroid[1]
        temp = np.sqrt(np.multiply(r1,r1) + np.multiply(r2,r2) - np.multiply(np.multiply(np.multiply(2,r1),r2),np.cos(theta2-theta1)))
        if(i==0):
            minDist = temp
        else:
            minDist = np.vstack((minDist,temp))

    currentFrame = np.empty((2,max(centpredCol,rthetaCol)))
    currentFrame[:] = np.nan

    minDist = np.reshape(minDist, (centpredCol,rthetaCol))
    minDistOrg = minDist

    for i in list(range(0,min(centpredCol,rthetaCol))):
        if((np.ndim(minDist)) == 1):
            minDist = np.reshape(minDist,(rthetaCol,1))
            minDistOrg = np.reshape(minDistOrg,(rthetaCol,1))
        val = np.min(minDist)
        resultOrg = np.argwhere(minDistOrg == val)
        result = np.argwhere(minDist == val)
        minRowOrg = resultOrg[0][0]
        minColOrg = resultOrg[0][1]
        minRow = result[0][0]
        minCol = result[0][1]
        currentFrame[:,minRowOrg] = rthetacentroid[:,minColOrg]
        minDist = np.delete(minDist,minRow,0)
        minDist = np.delete(minDist,minCol,1)
        rthetacentroidCurrent = np.delete(rthetacentroidCurrent,minCol,1)

    index = 0
    if (rthetacentroidCurrent.size != 0): #check indexing
        for i in list(range(centpredCol,rthetaCol)):
            currentFrame[:,i] = rthetacentroidCurrent[:,index]
            index += 1 

    return(currentFrame)

def LiveRKF(currentrawxycentroidData, centroidX, centroidP):
    
    #initialise matrices 
    delT = 0.0500
    A = np.array([[1,delT,0,0], 
                  [0,1,0,0], 
                  [0,0,1,delT], 
                  [0,0,0,1]])
    H = np.array([[1,0,0,0],
                  [0,0,1,0]])
    P = np.identity(4)
    Q = np.multiply(0.9,np.identity(4))
    R = np.array([[1],[1]])

    xytransposecentroidData = currentrawxycentroidData
    rthetacentroidData=xytransposecentroidData
    if (xytransposecentroidData.size != 0): 
        [rthetacentroidData[0,:],rthetacentroidData[1,:]] = cart2pol(xytransposecentroidData[0,:],xytransposecentroidData[1,:])
    if((rthetacentroidData.size != 0)):
        currentFrame = data_associate(centroidX, rthetacentroidData)
        addittionalCentroids = (np.size(rthetacentroidData,1)-np.size(centroidX,1))
        if(addittionalCentroids>0):
            centroidX = np.pad(centroidX, ((0,0),(0,addittionalCentroids)), 'constant') #initialises previous iteration to zer
            for newFrameIndex in list((range(0, addittionalCentroids))):
                centroidP.extend([P])
        for currentFrameIndex in list((range(0,np.size(currentFrame,1)))):
            if(not(np.isnan(currentFrame[0,currentFrameIndex]))):
                [xpred, Ppred] = predict(centroidX[:,currentFrameIndex], centroidP[currentFrameIndex], A, Q)
                [nu, S] = innovation(xpred, Ppred, currentFrame[:, currentFrameIndex], H, R)
                [centroidX[:,currentFrameIndex],  centroidP[currentFrameIndex]] = innovation_update(xpred, Ppred, nu, S, H)
            else:
                [centroidX[:,currentFrameIndex], centroidP[currentFrameIndex]] = predict(centroidX[:,currentFrameIndex], centroidP[currentFrameIndex], A, Q)                   
    else:
        for noFrameIndex in list((range(0,np.size(centroidX,1)))):
            [centroidX[:,noFrameIndex], centroidP[noFrameIndex]] = predict(centroidX[:,noFrameIndex], centroidP[noFrameIndex], A, Q)
            
    #centroidX is 4xN array that contains that centroid information for that frame
    return centroidX, centroidP


### Offline Code

In [22]:
parsingMatFile = 'C:\\Users\\hasna\\Documents\\GitHub\\OccupancyDetection\\Data\\Experiment Data 2\\2PeopleWalkingLocation.mat'
# parsingMatFile = 'C:\\Users\\hasna\\Documents\\GitHub\\OccupancyDetection\\Data\\Matlab Data\\fastWalk.mat'

tlvData = (loadmat(parsingMatFile))['tlvStream'][0]

centroidX =np.zeros((4,1))
centroidP = []
P = np.identity(4);
centroidP.extend([P])

snrThreshold = 25
usedFrames = 0

eps = 1.2
minSamples = 13

for tlvStream in tlvData:
    #parse
    posX, posY, SNR = liveParsing(tlvStream)

    try:
        if (posX == None).all():
            continue
    except AttributeError:
        continue
        
    clusters = DBSCANOnly(posX, posY, eps, minSamples)
    
    if clusters.size > 0:
        if int(max(clusters[2,:])) > 1:
            print('HELLO')
            break
        #row 1 - x
        #row 2 - y
        #row 3 - cluster number
        s1.setData(clusters[0,:], clusters[1,:])
        
        
        #k-mediods
        # Set random initial medoids.
        intialMediods = [1]
        #Set k and gather points
        k = int(max(clusters[2,:])) + 1 
        points = np.transpose(np.array([clusters[0,:], clusters[1,:]]))
        D = pairwise_distances(points, metric='euclidean')
        M, C = KMedoids.kMedoids(D, k) #M is the medoid index numbers
        medoids = points[M]
        s2.setData(medoids[:,0], medoids[:,1])
        QtGui.QApplication.processEvents() 
        time.sleep(0.05)
        

In [89]:
np.transpose(np.array([clusters[0,:], clusters[1,:]])).shape

(17, 2)

In [15]:
medoids.shape

(1, 2)